In [1]:
import pandas as pd
import numpy as np
import tensorflow

### Data Exploration

In [6]:
reviews = pd.read_json('/content/drive/MyDrive/Colab Notebooks/FlavorFriend/Data/yelp_academic_dataset_review.json', chunksize=10000, lines=True)
'''
get columns: 
Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')
'''

# for chunk in reviews:
#     print(chunk.columns)

    

"\nget columns: \nIndex(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',\n       'cool', 'text', 'date'],\n      dtype='object')\n"

In [2]:
users_chunk = pd.read_json('/content/drive/MyDrive/Colab Notebooks/FlavorFriend/Data/yelp_academic_dataset_user.json', chunksize=10000, lines=True)
users = pd.concat(users_chunk, ignore_index= True)

'''
columns = Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')
'''
# print(users)
# print(len(users))
# for chunk in users:
#     print(chunk.columns)

"\ncolumns = Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',\n       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',\n       'compliment_more', 'compliment_profile', 'compliment_cute',\n       'compliment_list', 'compliment_note', 'compliment_plain',\n       'compliment_cool', 'compliment_funny', 'compliment_writer',\n       'compliment_photos'],\n      dtype='object')\n"

In [3]:
print(len(users))

print(len(users['user_id'].unique()))

1987897
1987897


In [13]:
bus = pd.read_json('/content/drive/MyDrive/Colab Notebooks/FlavorFriend/Data/yelp_academic_dataset_business.json', chunksize=10000, lines=True)
bus = pd.DataFrame(bus)
'''
columns = Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')
'''

print(len(bus))
# for chunk in bus:
#     print(chunk.columns)

16


/usr/local/lib/python3.9/dist-packages/pandas/core/internals/construction.py:568: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


### Data Cleaning

In [2]:
class Yelp():

  def __init__(self):

    reviews_chunk = pd.read_json('/content/drive/MyDrive/Colab Notebooks/FlavorFriend/Data/yelp_academic_dataset_review.json', chunksize=10000, lines=True)
    self.reviews = pd.concat(reviews_chunk, ignore_index= True)

    users_chunk = pd.read_json('/content/drive/MyDrive/Colab Notebooks/FlavorFriend/Data/yelp_academic_dataset_user.json', chunksize=10000, lines=True)
    self.users = pd.concat(users_chunk, ignore_index= True)

    business_chunk = pd.read_json('/content/drive/MyDrive/Colab Notebooks/FlavorFriend/Data/yelp_academic_dataset_business.json', chunksize=10000, lines=True)
    self.business = pd.concat(business_chunk, ignore_index= True)

    '''
    Columns: 

    1. Reviews
      ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date']

    2. Users
        ['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos']

    3. Business
        ['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours']
    '''


  def Clean_Business(self, review_threshold):
    '''
    Return 
    1. restuarants only df
    2. restuarant counts
    3. total restuarant reviews
    '''

    count = 0
    drop_index = []
    df = self.business

    for index, row in df.iterrows():
        if row["categories"] is not None:
            if ("Restaurants" in (row["categories"]) or "Food" in (row["categories"])) and row["is_open"] == 1 and row["review_count"] >= review_threshold:
                count+=1
            else:
                drop_index.append(index)
        else:
            drop_index.append(index)

    df.drop(drop_index, axis=0, inplace=True)
    total_reviews = np.sum(list(df["review_count"]))
    
    return count, df, total_reviews


  
  def Clean_Reviews(self, cleaned_business):
    '''
    cleaned_business: pd.Dataframe()

    Return:
    1. user_lookup: {user_id : {'count' : reviewcount, 'names' : [list of bus]}}
    '''

    restuarant_ids = cleaned_business['business_id'].unique()
    reviews = self.reviews
    filter_review = reviews.loc[reviews['business_id'].isin(restuarant_ids)]

    user_lookup = {}

    for index, row in filter_review.iterrows():

      user_id = row['user_id']
      biz_id = row['business_id']

      if user_id not in user_lookup.keys():

        user_lookup[user_id] = {'count' : 1, 'names' : [biz_id]}

      else:
        user_lookup[user_id]['count'] += 1
        user_lookup[user_id]['names'].append(biz_id)
          

    return user_lookup




  def Reduce_Users(user_lookup, reviewcount):

    return 


### Test

In [3]:
yp = Yelp()

_, bu_df, _ = yp.Clean_Business(review_threshold= 15)

user_lookup = yp.Clean_Reviews(bu_df)

In [6]:
# print(len(user_lookup))

# count = 0

# for i in user_lookup:
#   if count < 10:
#     print(count, i)
#     count += 1

In [4]:
print(user_lookup['mh_-eMZ6K5RLWhZyISBhwA'])

print(len(user_lookup['mh_-eMZ6K5RLWhZyISBhwA']['names']))

{'count': 20, 'names': ['XQfwVwDr-v0ZS3_CbbE5Xw', '8eDkw7CE0NKqMknPIu26fw', 'tVfJPW14AeuAHDJeleWWdQ', 'mOk3D7VczrDapNuUgLxUQw', 'L4kfcADLCU4T33i7Z0CkuA', 'gpYBhnTk4KzvvH83TsZiQg', 'QWF_PoD4PthsYsq2Tk0GEQ', 'CjNjiHsyV-gvtOsHZB0VJQ', 'rQ1t0zD_TBTqCF06By_UgA', 'VvqYQ98FjO0iYpKHgu91fw', 'sYgyAxvuDP1799oiGXqE_A', 'ol5npt2HXo9r3yKa3wBlYQ', 'A0QbndhHwlqfY0McSy-xQA', 'XsvxRd2u8iRD_S50ZJ5-QQ', 'NnXtMdEJ4QRgaw7hTgbkgg', 'WL-0PLW5IzdnyUHGmiOrgQ', 'QGzyGNkNBnB1q6KgeUL3Cg', 'HpCsADSPPSoCghfcZeT55w', '-rwfGlw6T5czqQO4uZGWYw', 'rAe-1HU5Z-DuUXEbzASXDA']}
20


In [ ]:
count_list = [num for user_lookup[num]]